# Dataset cleaning

Our main goal in this notebook is to turn the level 1 data provided in the [main dataset](https://www.spaceappschallenge.org/develop-the-oracle-of-dscovr-experimental-data-repository/) into level 2 data through data processing and feature engineering.

In [2]:
import pandas as pd
import numpy as np
import pycatch22
import json
import pdb
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, cross_val_score
import ray

import warnings
warnings.filterwarnings(action='ignore')

import os
import gc
import pdb
import keras
import urllib3
import tensorflow
from sklearn import metrics
from xgboost import XGBClassifier
from keras.models import load_model
from bayes_opt import BayesianOptimization
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import math

Here, we read the .csv files that are in the same folder than this jupyter notebook:

In [6]:
i = 2016
dataDSCOVR = pd.read_csv('dsc_fc_summed_spectra_'+str(i)+'_v01.csv',delimiter = ',', parse_dates=[0], \
infer_datetime_format=True, na_values='0', \
header = None)

for i in range(2017, 2024):
  aux = pd.read_csv('dsc_fc_summed_spectra_'+str(i)+'_v01.csv',delimiter = ',', parse_dates=[0], \
  infer_datetime_format=True, na_values='0', \
  header = None)

  dataDSCOVR = pd.concat([dataDSCOVR, aux])

Changing the of all column features:

In [7]:
# dataDSCOVR.drop(dataDSCOVR.columns.difference([0,1, 2, 3]), 1, inplace=True)
new_cols = ['datetime', 'E_x', 'E_y', 'E_z']
new_cols += ['f_cup_'+str(i) for i in range(50)]
dataDSCOVR.columns = new_cols

and oppening the K$_p$ dataset from .json

In [8]:
f = open('kp.json')
kps = json.load(f)
kps = pd.DataFrame({'datetime': kps['datetime'], 'kp':kps["Kp"]})
kps.replace(['T', 'Z'], [' ',''], regex=True, inplace=True)
kps

,datetime,kp
0,2016-01-01 00:00:00,6.000
1,2016-01-01 03:00:00,5.333
2,2016-01-01 06:00:00,5.000
3,2016-01-01 09:00:00,3.333
4,2016-01-01 12:00:00,1.667
...,...,...
22683,2023-10-06 09:00:00,1.667
22684,2023-10-06 12:00:00,1.000
22685,2023-10-06 15:00:00,1.333
22686,2023-10-06 18:00:00,1.000


Reseting the index:

In [9]:
dataDSCOVR.reset_index()

,index,datetime,E_x,E_y,E_z,f_cup_0,f_cup_1,f_cup_2,f_cup_3,f_cup_4,...,f_cup_40,f_cup_41,f_cup_42,f_cup_43,f_cup_44,f_cup_45,f_cup_46,f_cup_47,f_cup_48,f_cup_49
0,0,2016-01-01 00:00:00,6.83609,-3.37934,-12.920500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2016-01-01 00:01:00,6.76732,-3.30194,-12.996700,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2016-01-01 00:02:00,6.39107,-2.61173,-13.327100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2016-01-01 00:03:00,6.44897,-2.61525,-13.329900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2016-01-01 00:04:00,6.58758,-2.73082,-13.236100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3277435,175675,2023-05-02 23:55:00,4.33665,2.80127,0.954633,1.004790,0.231726,4.64757,0.626576,3.54595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277436,175676,2023-05-02 23:56:00,4.66778,2.10582,1.260660,0.781621,0.231726,5.97222,0.360648,3.39143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277437,175677,2023-05-02 23:57:00,4.28322,3.02154,0.927773,0.854185,0.231726,8.10454,2.025800,4.22000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277438,175678,2023-05-02 23:58:00,4.31376,2.67727,1.723270,0.801559,0.231726,12.76210,2.818550,4.82242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Matching dates of K$_p$ and DSCOVR dataset

In [10]:
for idx, i in enumerate(kps['datetime'].iloc[:]):
    if i > str(dataDSCOVR['datetime'].iloc[-1]):
        kps.drop(idx, inplace=True)

dataDSCOVR.reset_index(drop=True, inplace=True)

for idx, i in enumerate(dataDSCOVR['datetime'].iloc[:]):
    if str(i) > str(kps['datetime'].iloc[-1]):
        dataDSCOVR.drop(idx, inplace=True)

Getting the value of K$_p$ per measasurement of main dataset:

In [15]:
indices = []
elements = []
ini_j = 0

for idx_i, i in enumerate(kps['datetime'].iloc[:]):
  if idx_i > 0:
    val = kps['kp'][idx_i]

    for idx_j in range(ini_j, len(dataDSCOVR['datetime'].iloc[:])):
      j_val = dataDSCOVR['datetime'].iloc[idx_j]
      if str(i) >= str(j_val):
        elements.append(val)
        indices.append(idx_i-1)
      else:
        break

    ini_j = idx_j


Setting the K$_p$ and the index in the same dataset. Here, when two or more measurements have the same index number, implies they happen in the same 3 hours interval.

In [16]:
dataDSCOVR['kp'] = elements
dataDSCOVR['idx'] = indices

Visualization of the data:

In [17]:
dataDSCOVR

,datetime,E_x,E_y,E_z,f_cup_0,f_cup_1,f_cup_2,f_cup_3,f_cup_4,f_cup_5,...,f_cup_42,f_cup_43,f_cup_44,f_cup_45,f_cup_46,f_cup_47,f_cup_48,f_cup_49,kp,idx
0,2016-01-01 00:00:00,6.83609,-3.37934,-12.920500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.333,0
1,2016-01-01 00:01:00,6.76732,-3.30194,-12.996700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.333,0
2,2016-01-01 00:02:00,6.39107,-2.61173,-13.327100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.333,0
3,2016-01-01 00:03:00,6.44897,-2.61525,-13.329900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.333,0
4,2016-01-01 00:04:00,6.58758,-2.73082,-13.236100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3277256,2023-05-02 20:56:00,4.39297,2.71839,-0.446718,0.618360,0.231726,12.42580,12.927500,6.16945,0.361102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.667,21430
3277257,2023-05-02 20:57:00,4.15341,2.81890,-1.188930,0.231726,0.231726,19.71780,24.135800,4.39875,0.231726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.667,21430
3277258,2023-05-02 20:58:00,3.68157,3.23417,-1.221200,2.203270,0.231726,14.45130,11.234300,3.25098,0.448420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.667,21430
3277259,2023-05-02 20:59:00,3.88026,2.68123,-1.736670,2.074320,0.231726,11.98320,12.397100,7.44632,0.353663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.667,21430


## Data Cleaning and Feature engeneering

Here, we catch and lines that has at least a null element in magnetommeter and drop out this lines.

In [ ]:
isnull_df = dataDSCOVR[dataDSCOVR.columns[0:4]].isnull()
any_nan = isnull_df.any(axis=1)
isnull_df = any_nan.to_numpy().nonzero()[0]
isnull_df = np.sort(isnull_df, axis=None)

indexes = []
for i in isnull_df:
  indexes.append(dataDSCOVR['idx'][i])
indexes.sort()

unique_indexes = []
unique_indexes.append(indexes[0])

for item in indexes:
  if item != unique_indexes[-1]:
    unique_indexes.append(item)

dataDSCOVR = dataDSCOVR[~dataDSCOVR['idx'].isin(unique_indexes)]

And verify if exist any line that all elements of Fadaray Cup measurements are null, and drop out this lines.

In [ ]:
mask = dataDSCOVR[dataDSCOVR.columns[4:-2]].isna().all(axis=1)
rows_with_all_nan = dataDSCOVR[mask].index.tolist()
dataDSCOVR.drop(rows_with_all_nan, inplace=True)
dataDSCOVR.fillna(0, inplace=True)
dataDSCOVR.reset_index(drop=True, inplace=True)

For Feature engeneering we will need to first define a numerical integration function:

In [45]:
def trapezoid(x,y):
    n = len(x)
    h = (x[n-1] - x[0])/n

    I = h*(y[0] + y[n-1])/2 + h*np.sum(y[2:n-1])
    return I

Finally, we will get the numerical values of Density, Velocity and Temeperature using the numerical values in Faraday Cups measurements.

In [ ]:
k = 1.380649* 10**(-23)
density = np.zeros(len(dataDSCOVR))
velocity = np.zeros(len(dataDSCOVR))
temperature = np.zeros(len(dataDSCOVR))

for i in range(len(dataDSCOVR)):
    aux = dataDSCOVR.iloc[i, 4:55]
    den_aux = trapezoid(range(50), aux) / np.linalg.norm(aux)

    if den_aux == 0:
        density[i] = den_aux
        velocity[i] = 0
        temperature[i] = 0
    else:
        density[i] = den_aux
        velocity[i] = np.linalg.norm(aux)/den_aux
        temperature[i] = (2/3)*(np.linalg.norm(aux)/(k*den_aux))

dataDSCOVR['Density'] = density
dataDSCOVR['Velocity'] = velocity
dataDSCOVR['Temperature'] = temperature

Now, we drop all Faraday Cups measurements, we already used the useful part of this values

In [ ]:
dataDSCOVR.drop(columns=dataDSCOVR.columns[4:-5], inplace=True)

Now, reordering the dataset's columns and add a Euclidian norm for magnetommeter measurements.

In [ ]:
dataDSCOVR = dataDSCOVR.iloc[:, list(range(0,4)) + list(range(6,9)) + list(range(4,6))]

matrix = dataDSCOVR[dataDSCOVR.columns[1:-2]].values.T
norm = [math.sqrt(sum(x**2 for x in row)) for row in zip(*matrix)]
dataDSCOVR['norm'] = norm

dataDSCOVR = dataDSCOVR.iloc[:, list(range(0,7)) + [9] + list(range(7,9))]

We add a funtion to normalize the values of Dataset for reducing the impact of Outliers in the K$_p$ regressor.

In [ ]:
def absolute_maximum_scale(series):
    return (series - series.min()) / (series.max() - series.min())

for col in dataDSCOVR.columns[1:-2]:
    dataDSCOVR[col] = absolute_maximum_scale(dataDSCOVR[col])

And, finnaly, we save the dataset!

In [ ]:
dataDSCOVR.to_csv('dataDSCOVR.csv', index=False)